In [30]:
import pandas as pd 
import numpy as np
data = pd.read_csv("newData.csv") 
data.dtypes

Temperature    object
Outlook        object
Humidity       object
Windy            bool
left            int64
dtype: object

In [31]:
temp5 = list(data.index.values)
numberOfPlus = 0
numberOfNeg = 0
for i in temp5:
    if str(data.at[i,"left"])==str(1):
        numberOfPlus += 1
    else:
        numberOfNeg += 1
print(numberOfPlus)
print(numberOfNeg)
ProPos = numberOfPlus/(numberOfPlus+numberOfNeg)
ProNeg = numberOfNeg/(numberOfPlus+numberOfNeg)
baseLineAccuracy = (ProPos*ProPos)+(ProNeg*ProNeg)
print(ProPos)
print(baseLineAccuracy)

9
5
0.6428571428571429
0.5408163265306123


In [32]:
train = data[:]
numberOfNodes = 0
class Node(object):
    #def __init__(self, label,name,parentAttrValue,nodeNumber,level):
    def __init__(self, examples, attributes, decisionAttribute, nodeNumber, parentDecisionAttribute, parentDecisionAttributeValue, decision, level):
        self.examples = examples
        self.attributes = attributes
        self.decisionAttribute = decisionAttribute 
        self.nodeNumber = nodeNumber
        self.children = []
        self.parentDecisionAttribute = parentDecisionAttribute
        self.parentDecisionAttributeValue = parentDecisionAttributeValue
        self.decision = decision
        self.level = level

    def add_child(self, obj):
        self.children.append(obj)
        
    def increment_nodeNumber(self):
        #nodeNumber = nodeNumber+1
        self.nodeNumber = self.nodeNumber + 1
        
    def __str__(self):
        #return self.label +"  "+ self.name+"  "+self.parentAttrValue+"Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)
        return "Decision attribute for this node is "+self.decisionAttribute+" decision "+ self.decision+" Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))

In [33]:
targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
AllAttrExpTar = Attributes[:]
print(Attributes)
examples = list(train.index.values) 
len(examples)

['Temperature', 'Outlook', 'Humidity', 'Windy']


14

In [34]:
def getEntropyOfExamples(examplesForCurAttr):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurAttr:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurAttr)
    PropOfNeg = NumberOfNegEx/len(examplesForCurAttr)
    return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

In [35]:
def getEntropyofAttribute(currentAttribute,examplesForCurAttr,targetAttribute):
    NumberOfexamplesForCurAttr = len(examplesForCurAttr)
    sum = 0.0
    fra1 = train.loc[examplesForCurAttr,[currentAttribute] ]
    currentAttributeValues = fra1[currentAttribute].unique()
    #currentAttributeList = fra1[currentAttribute]
    #unique2, counts2 = np.unique(currentAttributesList, return_counts=True)
    #dictOfAttr = dict(zip(unique2, counts2))
    #print("currentAttribute is "+currentAttribute+" and its values are")
    #print(currentAttributeValues)
    for i in currentAttributeValues:
    #for i in unique2:
        NumberOfExaForThisValue = 0
        for j in examplesForCurAttr:
            if str(train.at[int(j),currentAttribute]) == str(i):
                NumberOfExaForThisValue = NumberOfExaForThisValue+1
        if NumberOfExaForThisValue == 0:
            #print("yes i am here now")
            sum = sum+0
        else:
            NumberOfPosForThisValue = 0
            NumberOfNegForThisValue = 0
            for j in examplesForCurAttr:
                if str(train.at[int(j),targetAttribute]) == str(1) and str(train.at[int(j),currentAttribute]) == str(i):
                    NumberOfPosForThisValue = NumberOfPosForThisValue+1
                elif str(train.at[int(j),targetAttribute]) == str(0) and str(train.at[int(j),currentAttribute]) == str(i):
                    NumberOfNegForThisValue = NumberOfNegForThisValue+1
            PropOfPos = NumberOfPosForThisValue/NumberOfExaForThisValue
            PropOfNeg = NumberOfNegForThisValue/NumberOfExaForThisValue
            PropOfThisValue = NumberOfExaForThisValue/NumberOfexamplesForCurAttr
            if PropOfPos == 0:
                sum = sum + ((PropOfThisValue)*(-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))
            elif PropOfNeg == 0:
                sum = sum + ((PropOfThisValue)*(-1*PropOfPos*(np.log(PropOfPos)/np.log(2))))
            else:
                sum = sum + ((PropOfThisValue)*((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))))              
    return sum

In [36]:
def BestOfAttributes(Attributes,examplesForCurAttr,targetAttribute,method):
    if len(Attributes) == 1:
        return Attributes[0]
    elif method == 'Entropy':
        EntropyForAttributes = {}
        for i in Attributes:
            EntropyForAttributes[i] = getEntropyofAttribute(i,examplesForCurAttr,targetAttribute)
        #findining the minimum among entropy of attributes
        print("Attributes are ")
        print(Attributes)
        print("entropy of attributes")
        print(EntropyForAttributes)
        BestOfAttributes = min(EntropyForAttributes, key=EntropyForAttributes.get)
        return BestOfAttributes        
    elif method == 'Gini':
        pass
    elif method == 'misClasifi':
        pass
    return BestOfAttributes

In [64]:
#(self, examples, attributes, decisionAttribute, nodeNumber, parentDecisionAttribute, parentDecisionAttributeValue, decision, level):
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,AttributesForCurrentNode,parentDecisionAttribute,parentAttrValue,level):
    # if all examplesForCurNode are positive(i.e employee left the company,left =1 )
    #if len(examplesForCurNode) == 0:
        #print('Yes I am here now')
     #   r = 
    global numberOfNodes
    #level
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:
        r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Positive',level)
        numberOfNodes  = numberOfNodes + 1
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        print("********************************************************************")    
        return r
    
    # if all examplesForCurNode are negative(i.e employee didnot leave the company,left =0 )
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Negative',level)
        numberOfNodes  = numberOfNodes + 1
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        #print("number of examplesForCurNode of this attribute is ")
        #print(len(examplesForCurNode))
        #print(examplesForCurNode)
        print("Type2")
        print("********************************************************************")
        return r
    
    #if attributes are empty , then most common value of target Attribute in examplesForCurNode
    if len(AttributesForCurrentNode) == 0 or getEntropyOfExamples(examplesForCurNode) < 0.01:#or len(examplesForCurNode)<50 :
        print("entropy of current examples ")
        print(str(getEntropyOfExamples(examplesForCurNode)))
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.5:
            r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Positive',level)
            numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Negative',level)
            numberOfNodes  = numberOfNodes + 1
        print("********************************************************************")
        print("The label of current node is")
        print(r)
        #print("number of examplesForCurNode of this attribute is ")
        #print(len(examplesForCurNode))
        #print(examplesForCurNode)
        #print("PosCount "+str(posCount)+"and "+str(negCount)+" negCount")
        print("Type3")
        print("********************************************************************")
        return r
    #if len(Attributes) == 0:
    #else:
        #Attributes = Attributes + AllAttrExpTar
    print("length of examplesForCurNode is "+str(len(examplesForCurNode)))
    BestAttribute = BestOfAttributes(AttributesForCurrentNode,examplesForCurNode,targetAttribute,'Entropy')    
    r = Node(examplesForCurNode,AttributesForCurrentNode,BestAttribute,numberOfNodes,parentDecisionAttribute,parentAttrValue,'NotLeaf',level)
    #Attributes.remove(BestAttribute)
    print("BestAttribute is")
    print(BestAttribute)
    numberOfNodes  = numberOfNodes + 1
    BestAttributeValues = list(data[BestAttribute].unique())
    print("BestAttributeValues are")
    print(BestAttributeValues)
    for iter in BestAttributeValues:
        AttributesForChildren = []
        AttributesForChildren = AttributesForCurrentNode[:]
        AttributesForChildren.remove(BestAttribute)
        examplesForThisChildOfCurrentNode = []
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(iter):
                examplesForThisChildOfCurrentNode.append(int(iter1))
        print("examplesForThisChildOfCurrentNode is "+str(len(examplesForThisChildOfCurrentNode)))
        if len(examplesForThisChildOfCurrentNode)==0:
            numberOfNodes  = numberOfNodes + 1
            fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
            targetAttributesList = fra1[targetAttribute].values
            unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
            dictOfTaAttr = dict(zip(unique1, counts1))
            posCount = dictOfTaAttr[1]
            negCount = dictOfTaAttr[0]
            if (posCount/(posCount+negCount))>0.5:
                r.add_child(Node(examplesForThisChildOfCurrentNode,AttributesForChildren,'leafNode',numberOfNodes,BestAttribute,str(iter),'Positive',level+1))
            else:
                r.add_child(Node(examplesForThisChildOfCurrentNode,AttributesForChildren,'leafNode',numberOfNodes,BestAttribute,str(iter),'Negative',level+1))                            
            print("********************************************************************")
            print("The label of current node is")
            #r.children[-1].increment_nodeNumber( )
            print(r.children[-1])
            print("Type5")
            print("********************************************************************")
            numberOfNodes = numberOfNodes - 1
        else:        
            r.add_child(DecisionTreeBuilder(examplesForThisChildOfCurrentNode,targetAttribute,AttributesForChildren,BestAttribute,str(iter),level+1))
    print("********************************************************************")
    print("The label of current node is")
    print(r)
    #print("the best decision attribute of current node is")
    #print(BestAttribute)
    #print("number of examplesForCurNode of this attribute is ")
    #print(len(examplesForCurNode))
    #print(examplesForCurNode)
    #print("children of current nodes")
    plabo = range(0,len(r.children))
    for k in plabo:
        pass
        #print(r.children[k])
    print("Type4")
    print("********************************************************************")
            
    return r

In [65]:
numberOfNodes = 0
examplesForRootNode = examples[:]
AttributesForRootNode = Attributes[:]
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,AttributesForRootNode,'root','root',0)
numberOfNodes

length of examplesForCurNode is 14
Attributes are 
['Temperature', 'Outlook', 'Humidity', 'Windy']
entropy of attributes
{'Temperature': 0.8760473218921718, 'Outlook': 0.6935361388961918, 'Humidity': 0.7884504573082896, 'Windy': 0.8921589282623617}
BestAttribute is
Outlook
BestAttributeValues are
['sunny', 'overcast', 'rain']
examplesForThisChildOfCurrentNode is 5
length of examplesForCurNode is 5
Attributes are 
['Temperature', 'Humidity', 'Windy']
entropy of attributes
{'Temperature': 0.4, 'Humidity': 0.0, 'Windy': 0.9509775004326937}
BestAttribute is
Humidity
BestAttributeValues are
['high', 'normal']
examplesForThisChildOfCurrentNode is 3
********************************************************************
The label of current node is
Decision attribute for this node is leafNode decision Negative Node number is 2 of level 2 number of examples of this node 3
Type2
********************************************************************
examplesForThisChildOfCurrentNode is 2
************

8

In [66]:
print(RootNode.decisionAttribute)
print(RootNode.attributes)
print(RootNode.children[2].parentDecisionAttributeValue)
print(RootNode.children[2].children[1])
print(RootNode.children[2].children[1].parentDecisionAttributeValue)

Outlook
['Temperature', 'Outlook', 'Humidity', 'Windy']
rain
Decision attribute for this node is leafNode decision Negative Node number is 7 of level 2 number of examples of this node 2
True


In [67]:
trainCopy = train.copy()
newColumn1 = [0]*len(trainCopy)
trainCopy['predict'] = newColumn1
trainCopy.head()
trainExamples = list(train.index.values)

In [73]:
def predict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        #print("Hello")
        #print(r.decision)
        if str(r.decision) == 'Positive':
            return 1
        else:
            #print("I am here")
            return '0'
    else:
        AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])
        #print(AttrValue)
        for j in r.children:
            if str(j.parentDecisionAttributeValue) == str(AttrValue):
                break
        #print(j.parentDecisionAttributeValue)
        if str(j.parentDecisionAttributeValue) == str(AttrValue):
            pass
            #print("its fine")
        else:
            print("nope its not")
        return predict(i,j,TestSet)

In [74]:
print(predict(2,RootNode,trainCopy))


1


In [50]:
trainCopy

,Temperature,Outlook,Humidity,Windy,left,predict
0,hot,sunny,high,False,0,0
1,hot,sunny,high,True,0,0
2,hot,overcast,high,False,1,0
3,cool,rain,normal,False,1,0
4,cool,overcast,normal,True,1,0
5,mild,sunny,high,False,0,0
6,cold,sunny,normal,False,1,0
7,mild,rain,normal,False,1,0
8,mild,sunny,normal,True,1,0
9,mild,overcast,high,True,1,0


In [75]:
TestSet = trainCopy
for i in list(TestSet.index.values):
    #valCopy.at[int(i),'predict'] = predict(i,RootNode)
    #TestSet.at[i,'predict'] = predict(i,RootNode,TestSet)
    print(predict(i,RootNode,TestSet))

0
0
1
1
1
0
1
1
1
1
1
0
0
1
